In [1]:
import itertools
import numpy as np
import pandas as pd
from cppy import *
from cppy.model_tools.to_cnf import *

In [2]:
# Relation between 'rows' and 'cols', Boolean Variables in a pandas dataframe
class Relation(object):
    # rows, cols: list of names
    def __init__(self, rows, cols):
        self.cols = cols
        self.rows = rows
        rel = BoolVar((len(rows), len(cols)))
        self.df = pd.DataFrame(index=rows, columns=cols)
        for i,r in enumerate(rows):
            for j,c in enumerate(cols):
                self.df.loc[r,c] = rel[i,j]
    # use as: rel['a','b']
    def __getitem__(self, key):
        try:
            return self.df.loc[key]
        except KeyError:
            return False


def exactly_one(lst):
    # return sum(lst) == 1
    # (l1|l2|..|ln) & (-l1|-l2) & ...
    allpairs = [(-a|-b) for a, b in itertools.combinations(lst, 2)]
    return [any(lst)] + allpairs


def exactly_one_at_most(lst):
    allpairs = [(-a|-b) for a, b in itertools.combinations(lst, 2)]
    return any(lst), allpairs

In [3]:
if True:
    """
    Logic grid puzzle: 'pasta' in CPpy
    Based on... to check originally, currently part of ZebraTutor
    Probably part of Jens Claes' master thesis, from a 'Byron...' booklet
    """
    # type1 = sauce
    # type2 = pasta
    # type3 = differences between values of type dollar
    # type4 = differences between values of type dollar
    # type5 = differences between values of type dollar
    dollar = ['4', '8', '12', '16']
    person = ['angie', 'damon', 'claudia', 'elisa']
    sauce = ['the_other_type1', 'arrabiata_sauce', 'marinara_sauce', 'puttanesca_sauce'] # type1
    pasta = ['capellini', 'farfalle', 'tagliolini', 'rotini']  # type2

    types = [dollar, person, sauce, pasta]
    n = len(types)
    m = len(types[0])
    assert all(len(types[i]) == m for i in range(n)), "all types should have equal length"

    chose = Relation(person, sauce)
    paid = Relation(person, dollar)
    ordered = Relation(person, pasta)
    sauce_dollar = Relation(sauce, dollar) # is_linked_with_1(sauce, dollar)
    sauce_pasta = Relation(sauce, pasta) # is_linked_with_2(sauce, pasta)
    dollar_pasta = Relation(dollar, pasta) # is_linked_with_3(dollar, pasta)
    rels = [chose, paid, ordered, sauce_dollar, sauce_pasta, dollar_pasta]

    # Bijectivity
    cnt = 0
    bij = []
    bv_bij = []

    for rel in rels:
        # bijection for all columns inside relation
        bv1 = BoolVar()
        bv2 = BoolVar()
        # for each relation
        for col_ids in rel.df:
            # one per column
            atleast, atmost = exactly_one_at_most(rel[:, col_ids])
            [bij.append(implies(bv1, clause)) for clause in atmost]
            bij.append(implies(bv2, atleast))
        bv_bij.append(bv1)
        bv_bij.append(bv2)

        # bijection for all rows inside relation
        bv3 = BoolVar()
        bv4 = BoolVar()
        for (_,row) in rel.df.iterrows():
            # one per row
            atleast, atmost = exactly_one_at_most(row)
            [bij.append(implies(bv3, clause)) for clause in atmost]
            bij.append(implies(bv4, atleast))
        bv_bij.append(bv3)
        bv_bij.append(bv4)

    # Transitivity
    trans = []
    bv_trans =  [BoolVar() for i in range(12)]


    for x in person:
        for y in sauce:
            for z in dollar:
                t0 = to_cnf(implies(paid[x, z] & sauce_dollar[y, z], chose[x, y]))
                [trans.append(implies(bv_trans[0], clause)) for clause in t0]

                t1 = to_cnf(implies(~paid[x, z] & sauce_dollar[y, z], ~chose[x, y]))
                [trans.append(implies(bv_trans[1], clause)) for clause in t1]

                t2 = to_cnf(implies(paid[x, z] & ~sauce_dollar[y, z], ~chose[x, y]))
                [trans.append(implies(bv_trans[2], clause)) for clause in t2]

    for x in person:
        for y in sauce:
            for z in pasta:
                t3 = to_cnf(implies(ordered[x, z] & sauce_pasta[y, z], chose[x, y]))
                [trans.append(implies(bv_trans[3], clause)) for clause in t3]

                t4 = to_cnf(implies(~ordered[x, z] & sauce_pasta[y, z], ~chose[x, y]))
                [trans.append(implies(bv_trans[4], clause)) for clause in t4]

                t5 = to_cnf(implies(ordered[x, z] & ~sauce_pasta[y, z], ~chose[x, y]))
                [trans.append(implies(bv_trans[5], clause)) for clause in t5]

    for x in person:
        for y in dollar:
            for z in pasta:
                t6 = to_cnf(implies(ordered[x, z] & dollar_pasta[y, z], paid[x, y]))
                [trans.append(implies(bv_trans[6], clause)) for clause in t6]

                t7 = to_cnf(implies(~ordered[x, z] & dollar_pasta[y, z], ~paid[x, y]))
                [trans.append(implies(bv_trans[7], clause)) for clause in t7]

                t8 = to_cnf(implies(ordered[x, z] & ~dollar_pasta[y, z], ~paid[x, y]))
                [trans.append(implies(bv_trans[8], clause)) for clause in t8]


    for x in sauce:
        for y in dollar:
            for z in pasta:
                t9 = to_cnf(implies(sauce_pasta[x, z] & dollar_pasta[y, z], sauce_dollar[x, y]))
                [trans.append(implies(bv_trans[9], clause)) for clause in t9]

                t10 = to_cnf(implies(~sauce_pasta[x, z] & dollar_pasta[y, z], ~sauce_dollar[x, y]))
                [trans.append(implies(bv_trans[10], clause)) for clause in t10]

                t11 = to_cnf(implies(sauce_pasta[x, z] & ~dollar_pasta[y, z], ~sauce_dollar[x, y]))
                [trans.append(implies(bv_trans[11], clause)) for clause in t11]

    clues = []
    bv_clues = [BoolVar() for i in range(8)]

    # 0.The person who ordered capellini paid less than the person who chose arrabiata sauce
    # assumption_satisfied( 0  ) => ?a [person] b [type3] c [dollar] d [person] e [dollar]: ordered(a,capellini) & b>0 & chose(d,arrabiata_sauce) & paid(d,c) & e = c-b & paid(a,e).
    clue0 = to_cnf(any([ordered[a, "capellini"] & chose[d, "arrabiata_sauce"] & paid[d, c] & paid[a, e]
                for a in person
                for b in [-4, 4, -8, 8,  -12, 12]
                for c in dollar
                for d in person
                for e in dollar if (b > 0) and (int(e) == int(c)-b)]))
    [clues.append(implies(bv_clues[0], cl)) for cl in clue0 ]

    # 1. The person who chose arrabiata sauce ordered farfalle
    # assumption_satisfied( 0  ) => ?f [person]: chose(f,arrabiata_sauce) & ordered(f,farfalle).
    clue1 = to_cnf( any( [ chose[f,  "arrabiata_sauce"] & ordered[f, "farfalle"] for f in person]))
    [clues.append(implies(bv_clues[1], cl)) for cl in clue1 ]

    # assumption_satisfied( 0  ) => !f [person]: chose(f,arrabiata_sauce) => ordered(f,farfalle).
    # was ok maar vertaling van iets anders:
    #c1a = to_cnf([implies(chose[p, "arrabiata_sauce"], ordered[p, "farfalle"]) for p in person])
    # [clues.append(implies(bv_clues[1], clause)) for clause in c1a]

    # 2. The person who ordered tagliolini paid less than the person who chose marinara sauce
    # assumption_satisfied( 0  ) => ?g [person] h [type4] i [dollar] j [person] k [dollar]: ordered(g,tagliolini) & h>0 & chose(j,marinara_sauce) & paid(j,i) & k = i-h & paid(g,k).
    c2a = []
    for g in person:
        for h in [-4, 4, -8, 8, -12, 12]:
            if h > 0:
                for i in dollar:
                    for j in person:
                        for k in dollar:
                            if int(k) == int(i) - h:
                                c2a.append(ordered[g, "taglioni"] & chose[j, "marinara_sauce"] & paid[j, i] & paid[g, k])
    c2a = to_cnf(any(c2a))
    [clues.append(implies(bv_clues[2], clause)) for clause in c2a]

    #  3. The person who ordered tagliolini paid more than Angie
    # assumption_satisfied( 0  ) => ?l [person] m [type5] n [dollar] o [dollar]: ordered(l,tagliolini) & m>0 & paid(angie,n) & o = n+m & paid(l,o).
    c3a = []
    for l in person:
        for m in [-4, 4, -8, 8, -12, 12]:
            if m > 0:
                for n in dollar:
                    for o in dollar:
                        if int(o) == int(n) + m:
                            c3a.append(ordered[l, "taglioni"] & paid["angie", n] & paid[l, o])
    c3a = to_cnf(any(c3a))
    [clues.append(implies(bv_clues[3], clause)) for clause in c3a]

    #  4. The person who ordered rotini is either the person who paid $8 more than Damon or the person who paid $8 less than Damon
    # assumption_satisfied( 0  ) => ?
    # p [person]: ordered(p,rotini) & 
    #           ((?q [person] r [dollar] s [dollar]: paid(damon,r) & s = r+8 & paid(q,s) & q = p) 
    #           | (?t [person] u [dollar] v [dollar]: paid(damon,u) & v = u-8 & paid(t,v) & t = p)).

    #list with: for every person: two options
    c4a = []
    for p in person:
        formule1 = any(
            [paid["damon", r] & paid[q,s] for q in person for r in dollar for s in dollar if (int(s) == int(r) - 8) and (q == p)]
        )
        formule2 = any(
            [paid["damon", r] & paid[q,s] for q in person for r in dollar for s in dollar if (int(s) == int(r) + 8) and (q == p)]
        )
        groteformule = formule1 | formule2
        c4a.append(ordered[p, "rotini"] & groteformule)
    c4a = to_cnf(any(c4a))

    for clause in c4a:
        clues.append(implies(bv_clues[4], clause))

    # 5. Claudia did not choose puttanesca sauce
    # assumption_satisfied( 0  ) => ~ chose(claudia,puttanesca_sauce).
    c5a = to_cnf(implies(bv_clues[5],  ~chose["claudia", "puttanesca_sauce"]))
    clues.append(c5a)

    #  6. The person who ordered capellini is either Damon or Claudia
    # assumption_satisfied( 0  ) => ?w [person]: ordered(w,capellini) & (damon = w | claudia = w).
    c6a = to_cnf(any([ordered[p, 'capellini'] & ( (p == 'claudia') | (p == 'damon')) for p in person]))
    [clues.append(implies(bv_clues[6], clause)) for clause in c6a]

    # 7. The person who chose arrabiata sauce is either Angie or Elisa => XOR
    # assumption_satisfied( 0  ) => ?x [person]: chose(x,arrabiata_sauce) & (angie = x | elisa = x).
    c7a = to_cnf(any([chose[p, 'arrabiata_sauce'] &  ( (p == 'angie') | (p == 'elisa'))  for p in person]))
    [clues.append(implies(bv_clues[7], clause)) for clause in c7a]

    clueTexts = [
        "The person who ordered capellini paid less than the person who chose arrabiata sauce",
        "The person who chose arrabiata sauce ordered farfalle",
        "The person who ordered tagliolini paid less than the person who chose marinara sauce",
        "The person who ordered tagliolini paid more than Angie",
        "The person who ordered rotini is either the person who paid $8 more than Damon or the person who paid $8 less than Damon",
        "Claudia did not choose puttanesca sauce",
        "The person who ordered capellini is either Damon or Claudia",
        "The person who chose arrabiata sauce is either Angie or Elisa"
    ]

    clues_cnf = cnf_to_pysat(to_cnf(clues))
    bij_cnf = cnf_to_pysat(to_cnf(bij))
    trans_cnf = cnf_to_pysat(to_cnf(trans))

    hard_clauses = [c for c in clues_cnf + bij_cnf + trans_cnf]
    soft_clauses = []
    soft_clauses += [[bv1.name + 1] for bv1 in bv_clues]
    soft_clauses += [[bv1.name + 1]  for bv1 in bv_bij]
    soft_clauses += [[bv1.name + 1]  for bv1 in bv_trans]

    weights = {}
    weights.update({bv.name + 1: 100 for bv in bv_clues})
    weights.update({bv.name + 1: 60 for bv in bv_trans})
    weights.update({bv.name + 1: 60 for bv in bv_bij})

    explainable_facts = set()
    bvRels = {}
    for rel, relStr in zip(rels, ["chose", "paid", "ordered", "sauce_dollar", "sauce_pasta", "dollar_pasta"]):
        rowNames = list(rel.df.index)
        columnNames = list(rel.df.columns)

        # production of explanations json file
        for r in rowNames:
            for c in columnNames:
                bvRels[rel.df.at[r, c].name + 1] = {"pred" : relStr.lower(), "subject" : r.lower(), "object": c.lower()}

        # facts to explain
        for item in rel.df.values:
            explainable_facts |= set(i.name+1 for i in item)

    #for rel in rels:
    #    rowNames = list(rel.df.index)
    #    columnNames = list(rel.df.columns)
    #    for r in rowNames:
    #        for c in columnNames:
    #            print(r, c, rel.df.at[r, c].name + 1)

    #print(explainable_facts)
    # true_facts = [67, 50, 9, 70, 56, 14, 73, 59, 7, 80, 61, 4]
    true_facts = []#[67, 50, 9, 70, 56, 14, 73]

    matching_table = {
        'bvRel': bvRels,
        'Transitivity constraint': [bv.name + 1 for bv in bv_trans],
        'Bijectivity': [bv.name + 1 for bv in bv_bij],
        'clues' : {
            bv.name + 1: clueTexts[i] for i, bv in enumerate(bv_clues)
        }
    }

    #return hard_clauses, soft_clauses, weights, explainable_facts, matching_table, [[l] for l in true_facts], rels


In [13]:
def check_clue(clue, m):
    cnf = cnf_to_pysat(clue)
    return check_cnf(cnf, m)
def check_cnf(cnf, m):
    for cl in cnf:
        if not any((l in m) for l in cl):
            return False
    return True
#print(check_clue(clue1, m))

In [15]:
# lets solve it...
# pysat imports
from pysat.formula import CNF
from pysat.solvers import Solver

cnf = hard_clauses + soft_clauses + [[l] for l in true_facts]
with Solver(bootstrap_with=cnf) as s:
    sat = s.solve()
    print(sat)
    prev = None
    for id, m in enumerate(s.enum_models()):
            print(f"{id}: model found")
            #visu_int2d(m, puzzle)
            #print(np.array(m).reshape(n,n,n))
            print("cl0", check_clue(clue0, m))
            print("cl1", check_clue(clue1, m))
            print("cl2", check_clue(c2a, m))
            print("cl3", check_clue(c3a, m))
            print("cl4", check_clue(c4a, m))
            print("cl5", check_clue(c5a, m))
            print("cl6", check_clue(c6a, m))
            print("cl7", check_clue(c7a, m))
            print("hard", check_cnf(hard_clauses, m))
            if not prev is None:
                print("diff", sorted(set(prev) - set(m), key=lambda l: abs(l)))
                break
            prev = m


True
0: model found
cl0 True
cl1 True
cl2 True
cl3 True
cl4 True
cl5 True
cl6 True
cl7 True
hard True
1: model found
cl0 True
cl1 True
cl2 True
cl3 True
cl4 True
cl5 True
cl6 True
cl7 True
hard True
diff [5, -7, -9, 11, -49, 50, 57, -58, -65, 67, 73, -75, -134, 137, 158, -161, -182, 184, -186, 206, -208, 210, -229, 231, 233, 253, -255, -257, -278, 281, 302, -305, -326, 332, 350, -356, -374, 379, -381, 398, -403, 405, -421, 423, 428, 445, -447, -452, -470, 476, 494, -500, -709, 710, 717, -723, 727, -728, -735, 741, -747, 753, 805, -806, -813, 819, -823, 824, 831, -837, 843, -849]


In [16]:
for cl in hard_clauses:
    if 5 in cl:
        print(cl)

[-98, 1, 5, 9, 13]
[-100, 5, 6, 7, 8]
[-121, -181, 5]
[-121, -184, 5]
[-121, -187, 5]
[-121, -190, 5]
[-124, -373, 5]
[-124, -376, 5]
[-124, -379, 5]
[-124, -382, 5]
